In [2]:
import pixy2
from threading import Thread, Event
from time import sleep, time
import pickle

In [6]:
pixy = pixy2.Pixy2I2C(address=0x54)
frames = []

In [1]:
def collect_until_stopped(event):
    pixy.set_mode(pixy2.LINE_MODE_WHITE_LINE)
    
    pixy.set_lamp(1)
    fps = pixy.get_fps()
    window = 1.0 / fps
    res = pixy.get_resolution()
    if res is None:
        print('failed getting the resolution')
        return
    frames = [[res[0], res[1]]]
    frames.append([fps])
    
    while not event.is_set():
        features = pixy.get_main_features(features=1)
        if features is not None:     
            if 'vectors' in features:
                frames.append(features['vectors'])
            else:
                frames.append([])
        sleep(window)
        
    pixy.set_lamp(0)
    
    with open('dataset/pixy2_vector_dataset.obj', 'wb') as fp:
        pickle.dump(frames, fp)

In [12]:
stop_event = Event()
thread = Thread(target=collect_until_stopped, args=(stop_event,))
thread.start()

In [13]:
stop_event.set()